# Machine Learning Validação de modelos

## Objetivos
* Entenda os perigos do hold out
* Aplique técnicas de validação cruzada (cross validation)
* Utilize a aleatoriedade a seu favor
* Entenda quando usar diversas estratégias diferentes de validação cruzada
* KFold, StratifiedKFold, GroupKFold
* Trabalhe na previsão de novos grupos quando nem todos os dados são leituras independentes entre si
* Utilize um pipeline para treino e validação


## Importando as bibliotecas necessárias

In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate, KFold, StratifiedKFold, GroupKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

## Importando o dataset e explorando os dados

In [7]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

df = pd.read_csv(uri, index_col=0)
df.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [80]:
print(f'Esses dados possuem {df.shape[0]} amostras e {df.shape[1]} atributos')

Esses dados possuem 10000 amostras e 4 atributos


In [10]:
X = df.drop('vendido', axis=1)
y = df['vendido']

seed = 158020
np.random.seed(seed)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

### Baseline

In [14]:
seed = 158020
np.random.seed(seed)

dummy = DummyClassifier()
dummy.fit(X_train, y_train)
accuracy = dummy.score(X_test, y_test)

print(f'A Acurácia do modelo dummy stratified é: {accuracy:0.2%}')

A Acurácia do modelo dummy stratified é: 58.00%


### Decision Tree

In [16]:
seed = 158020
np.random.seed(seed)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(X_train, y_train)
accuracy = modelo.score(X_test, y_test)

print(f'A Acurácia do modelo dummy stratified é: {accuracy:0.2%}')

A Acurácia do modelo dummy stratified é: 71.92%


### Cross Validation

In [22]:
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv=3)
results['test_score']

array([0.75704859, 0.7629763 , 0.75337534])

In [23]:
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv=3)
results['test_score'].mean()

0.7578000751484867

In [59]:
def imprime_resultados(results, cv):
    results_media = results['test_score'].mean()
    results_std = results['test_score'].std()
    print(f'A média é das acurácias é: {results_media:0.2%}')
    print(f'A Acurácia do modelo para cv = {cv} é: [{results_media - 2 * results_std:0.2%} {results_media + 2 * results_std:0.2%}]')

In [60]:
SEED = 301
np.random.seed(SEED)

k = 3
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv=k)
imprime_resultados(results, k)

A média é das acurácias é: 75.78%
A Acurácia do modelo para cv = 3 é: [74.99% 76.57%]


In [61]:
SEED = 301
np.random.seed(SEED)

k = 5
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv=k)
imprime_resultados(results, k)

A média é das acurácias é: 75.78%
A Acurácia do modelo para cv = 5 é: [75.21% 76.35%]


In [62]:
SEED = 301
np.random.seed(SEED)

k = 10
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv=k)
imprime_resultados(results, k)

A média é das acurácias é: 75.78%
A Acurácia do modelo para cv = 10 é: [74.24% 77.32%]


### Aleatoriedade no cross validate

In [63]:
SEED = 301
np.random.seed(SEED)

k=10
cv = KFold(n_splits=k)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv=cv)
imprime_resultados(results, k)

A média é das acurácias é: 75.78%
A Acurácia do modelo para cv = 10 é: [74.37% 77.19%]


In [65]:
SEED = 301
np.random.seed(SEED)

k=10
cv = KFold(n_splits=k, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv=cv)
imprime_resultados(results, k)

A média é das acurácias é: 75.76%
A Acurácia do modelo para cv = 10 é: [73.26% 78.26%]


In [66]:
SEED = 301
np.random.seed(SEED)

k=10
cv = KFold(n_splits=k, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X, y, cv=cv)
imprime_resultados(results, k)

{'fit_time': array([0.00899792, 0.00899982, 0.0090003 , 0.00800157, 0.00900006,
        0.00900054, 0.00799966, 0.00899959, 0.00800109, 0.00799823]),
 'score_time': array([0.00200391, 0.00299931, 0.00299978, 0.00199938, 0.00200033,
        0.00200176, 0.00200129, 0.00200057, 0.00300169, 0.00300145]),
 'test_score': array([0.746, 0.773, 0.751, 0.762, 0.756, 0.759, 0.756, 0.753, 0.759,
        0.763])}

### Simulando um caso com muito azar onde os dados estão ordenados

In [72]:
df_azar = df.sort_values('vendido')
X_azar = df_azar.drop('vendido', axis=1)
y_azar = df_azar['vendido']

df_azar

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704
...,...,...,...,...
5491,71910.43,1,9,25778.40812
1873,30456.53,1,6,15468.97608
1874,69342.41,1,11,16909.33538
5499,70520.39,1,16,19622.68262


In [73]:
SEED = 301
np.random.seed(SEED)

k=10
cv = KFold(n_splits=k)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X_azar, y_azar, cv=cv)
imprime_resultados(results, k)

A média é das acurácias é: 57.84%
A Acurácia do modelo para cv = 10 é: [34.29% 81.39%]


### Resolvendo esse problema utilizando o metodo shuffle do KFold

In [74]:
SEED = 301
np.random.seed(SEED)

k=10
cv = KFold(n_splits=k, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X_azar, y_azar, cv=cv)
imprime_resultados(results, k)

A média é das acurácias é: 75.78%
A Acurácia do modelo para cv = 10 é: [72.30% 79.26%]


### Resolvendo esse problema utilizando o StratifiedKFold

In [78]:
SEED = 301
np.random.seed(SEED)

k=10
cv = StratifiedKFold(n_splits=k)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X_azar, y_azar, cv=cv)
imprime_resultados(results, k)

A média é das acurácias é: 75.78%
A Acurácia do modelo para cv = 10 é: [73.83% 77.73%]


In [77]:
SEED = 301
np.random.seed(SEED)

k=10
cv = StratifiedKFold(n_splits=k, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X_azar, y_azar, cv=cv)
imprime_resultados(results, k)

A média é das acurácias é: 75.78%
A Acurácia do modelo para cv = 10 é: [73.55% 78.01%]


### Utilizando o GroupKFold para fazer a validação por grupo

O objetivo é treinar o modelo e testar para modelos que não "estão" no dataset, por exemplo o modelo apenas existir no futuro e não agora enquanto treinamos o modelo

In [94]:
num_aleatorios = np.random.randint(-2,3, size=10000)
df['modelo'] = df['idade_do_modelo'] + num_aleatorios
df['modelo'] = df['modelo'] + abs(df['modelo'].min()) + 1
df.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,22
1,40557.96,1,20,12622.05362,20
2,89627.50,0,12,11440.79806,16
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,8


In [101]:
SEED = 301
np.random.seed(SEED)

k=10
cv = GroupKFold(n_splits=k)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, X_azar, y_azar, cv=cv, groups=df['modelo'])
imprime_resultados(results, k)

A média é das acurácias é: 75.76%
A Acurácia do modelo para cv = 10 é: [72.37% 79.15%]


### Cross validation com StandardScaler

In [117]:
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

modelo = SVC()
modelo.fit(X_train_scaled, y_train)
accuracy = modelo.score(X_test_scaled, y_test)

print(f'A Acurácia do modelo é: {accuracy:0.2%}')

A Acurácia do modelo é: 74.40%


In [112]:
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(X_azar)
X_azar_scaled = scaler.transform(X_azar)

k=10
cv = GroupKFold(n_splits=k)
modelo = SVC()
results = cross_validate(modelo, X_azar_scaled, y_azar, cv=cv, groups=df['modelo'])
imprime_resultados(results, k)

A média é das acurácias é: 76.79%
A Acurácia do modelo para cv = 10 é: [74.19% 79.39%]


## Utilizando um Pipeline

In [115]:
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])
pipeline

k=10
cv = GroupKFold(n_splits=k)
results = cross_validate(pipeline, X_azar, y_azar, cv=cv, groups=df['modelo'])
imprime_resultados(results, k)

A média é das acurácias é: 76.81%
A Acurácia do modelo para cv = 10 é: [74.18% 79.44%]
